# Notebook for visualizing trajectories in Porto

## ALL TRAJECTORIES

In [1]:
# Porto coordinates
P_MAX_LON = -8.57
P_MIN_LON = -8.66
P_MAX_LAT = 41.19
P_MIN_LAT = 41.14

In [2]:
import pandas as pd
import folium
import os
import ast
from tqdm import tqdm  # Import tqdm for the progress bar

# Define the input file path
input_file = "../dataset/porto/porto_raw.csv"

# Load the Porto dataset
try:
    # Read the polyline data from the CSV file
    data = pd.read_csv(input_file)

    # Prepare list to hold all coordinates
    all_coords = []

    # Iterate through each row and extract coordinates from the 'POLYLINE' column
    # Adding tqdm to show the progress of reading the rows
    for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing rows"):
        # Convert the polyline string to a list of coordinates
        polyline = ast.literal_eval(row['POLYLINE'])
        swapped_polyline = [(lat, lon) for lon, lat in polyline]
        all_coords.extend(swapped_polyline)  # Add the coordinates to the list

    # Create a map centered on Porto's coordinates
    map = folium.Map(location=[(P_MAX_LAT + P_MIN_LAT) / 2, (P_MAX_LON + P_MIN_LON) / 2], zoom_start=10)

    count = 0

    # Use tqdm to display a progress bar during iteration over all_coords
    for lat, lon in tqdm(all_coords[::100], total=len(all_coords[::100]), desc="Plotting points"):
        folium.CircleMarker([lat, lon], radius=1, weight=1).add_to(map)
        count += 1

    # Define the output file path
    output_file_path = os.path.join("output/porto", "porto_map_dotted_all_cordinates.html")

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    # Save the map
    map.save(output_file_path)

    print(f"Number of plotted points: {count}")
    print(f"Map saved to {output_file_path}")

except Exception as e:
    print(f"Error: {e}")


Plotting points: 100%|██████████| 834094/834094 [01:39<00:00, 8396.64it/s] 


Number of plotted points: 834094
Map saved to output/porto/porto_map.html


### Map with polylines trajectories

In [3]:
import folium
import os
import glob
from tqdm import tqdm


# Define the input directory path
input_dir = '/Users/thomasnitsche/Developer/skole/master/masteroppgave/dataset/porto/output/'

 # Create a map centered on Rome's approximate coordinates
porto_center_lat = (P_MAX_LAT + P_MIN_LAT) / 2
porto_center_lon = (P_MAX_LON + P_MIN_LON) / 2
porto_map = folium.Map(location=[porto_center_lat, porto_center_lon], zoom_start=12)


# Get all text files prefixed with R_
txt_files = glob.glob(os.path.join(input_dir, "P_*.txt"))

# Iterate through each file and extract coordinates
for txt_file in tqdm(txt_files, total=len(txt_files), desc="Processing trajectories"):
    coordinates = []
    with open(txt_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line:  # Check if the line is not empty
                try:
                    lat, lon = map(float, line.split(','))
                    coordinates.append((lat, lon))
                except ValueError as e:
                    print(f"Error parsing line: {line} - {e}")
    
    # Add a polyline connecting all the points in the trajectory
    if coordinates:
        folium.PolyLine(locations=coordinates, color='blue').add_to(porto_map)

# Define the output file path
output_file_path = os.path.join("output/porto", "porto_map_lined_trajectories.html")

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Save the map
porto_map.save(output_file_path)

print(f"Map saved to {output_file_path}")

Processing trajectories: 100%|██████████| 3000/3000 [00:00<00:00, 10600.92it/s]


Map saved to output/porto/porto_map_lined_trajectories.html


## HEATMAP

In [4]:
import os
import pandas as pd
import folium  # Importing Folium
from folium.plugins import HeatMap
import ast
# Reading the data
input_file_path = os.path.join("../dataset/porto", "porto.csv")
df = pd.read_csv(input_file_path, usecols=["POLYLINE"])

all_coords = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Convert the polyline string to a list of coordinates
    polyline = ast.literal_eval(row['POLYLINE'])
    swapped_polyline = [(lat, lon) for lon, lat in polyline]
    all_coords.extend(swapped_polyline)
    
center_lat = (P_MAX_LAT + P_MIN_LAT) / 2
center_lon = (P_MAX_LON + P_MIN_LON) / 2
# Create a Folium map at the average location
map = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Add a HeatMap layer to the map
HeatMap(all_coords, radius=16, blur=15).add_to(map)

# Save the map to an HTML file
map_path = os.path.join("output/porto", "porto_heatmap_v1.html")
map.save(map_path)

print("Heatmap saved to", map_path)

Heatmap saved to output/porto/porto_heatmap_v1.html


In [3]:
import folium
import os
import glob
from folium.plugins import HeatMap

# Define the input directory path
input_dir = '/Users/thomasnitsche/Developer/skole/master/masteroppgave/dataset/porto/output/'

 # Create a map centered on Rome's approximate coordinates
porto_center_lat = (P_MAX_LAT + P_MIN_LAT) / 2
porto_center_lon = (P_MAX_LON + P_MIN_LON) / 2
porto_map = folium.Map(location=[porto_center_lat, porto_center_lon], zoom_start=12)

# Get all text files prefixed with R_
txt_files = glob.glob(os.path.join(input_dir, "P_*.txt"))

# Collect all coordinates for the heatmap
heatmap_data = []

# Iterate through each file and extract coordinates
for txt_file in txt_files:
    with open(txt_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line:  # Check if the line is not empty
                try:
                    lat, lon = map(float, line.split(','))
                    heatmap_data.append([lat, lon])
                except ValueError as e:
                    print(f"Error parsing line: {line} - {e}")

# Add a heatmap layer
if heatmap_data:
    HeatMap(heatmap_data, radius=16, blur=15).add_to(porto_map)

# Define the output file path
output_file_path = os.path.join("output/porto", "porto_heatmap_v2.html")

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Save the map
porto_map.save(output_file_path)

print(f"Heatmap saved to {output_file_path}")

Heatmap saved to output/porto/porto_heatmap_v2.html
